JULIAN DAVIS' CONTRIBUTIONS WEEK 1 (found below)

First, I read in datasets.

The first dataframe contains country information on demographic and socio-economic.

The second dataframe is compiled form 5 different csv (where each csv represented a different year).  The final dataframe maps countries to their happiness score (for years 2015 to 2019).

In [16]:
import pandas as pd

df_unesco = pd.read_csv("https://raw.githubusercontent.com/erib3/unesco_data/master/DEMO_DS_29112019163028002.csv")

data_dir = "https://raw.githubusercontent.com/juliandavis7/301/master/"

df_2015 = pd.read_csv(data_dir + "2015.csv")
df_2016 = pd.read_csv(data_dir + "2016.csv")
df_2017 = pd.read_csv(data_dir + "2017.csv")
df_2018 = pd.read_csv(data_dir + "2018.csv")
df_2019 = pd.read_csv(data_dir + "2019.csv")
df_2015 = df_2015[["Country", "Happiness Score"]]
df_2016 = df_2016[["Country", "Happiness Score"]]
df_2017 = df_2017[["Country", "Happiness.Score"]]
df_2018 = df_2018[["Country or region", "Score"]]
df_2019 = df_2019[["Country or region", "Score"]]

df_2017.rename(columns=
        {"Happiness.Score": "Happiness Score"}, 
        inplace=True)
df_2018.rename(columns=
        {"Country or region": "Country", "Score": "Happiness Score"},
        inplace=True)
df_2019.rename(columns=
        {"Country or region": "Country", "Score": "Happiness Score"},
        inplace=True)

df_2015["Year"] = 2015
df_2016["Year"] = 2016
df_2017["Year"] = 2017
df_2018["Year"] = 2018
df_2019["Year"] = 2019

df_hap = pd.concat([df_2015, df_2016, df_2017, df_2018, df_2019])
df_hap.reset_index(drop=True, inplace=True)

In [25]:
len(df_unesco["Country"].unique())

233

In [26]:
len(df_hap["Country"].unique())

170

After reading in the data, we have to clean and reorganize the data.  I started by choosing for the indicators that we wanted to investigate further (and the ones that we though might be correlated with happiness).

In [17]:
indicators = ["Fertility rate, total (births per woman)",
              "Life expectancy at birth, total (years)",
              "Mortality rate, infant (per 1,000 live births)",
              "Population growth (annual %)",
              "Rural population (% of total population)",
              "GDP growth (annual %)",
              "GDP (current US$)",
              "GDP per capita (current US$)",
              "GDP per capita, PPP (current international $)",
              "GDP, PPP (current international $)",
              "GNI (current LCU)",
              "GNI per capita, Atlas method (current US$)",
              "GNI per capita, PPP (current international $)",
              "Population aged 14 years or younger ",
              "Population aged 15-24 years ",
              "Population aged 25-64 years ",
              "Population aged 65 years or older ",
              "Prevalence of HIV, total (% of population ages 15-49)",
              "Poverty headcount ratio at $1.90 a day (PPP) (% of population)",
              "Total population "]
#Comparing GNI to GDP shows the degree to which a nation's GDP represents domestic or international activity.

Oddly, the indicator data was stored in row format.  In this cell, we reorganize the dataframe such that each distinct row (observation) is a given country in a given year.

In [18]:
#new df with just one country and time match...
print(df_unesco.dtypes)
# we want this to have unique country and time values
# how long it take? 1970 -> 2019 * num indicators * num countries
# get all rows that match on country and time
final_df = df_unesco.copy()

num_unique_indicators = len(final_df["Indicator"].unique())
unique_indicators = final_df["Indicator"].unique()
for i in range(num_unique_indicators-1):
  final_df = final_df[final_df["Indicator"] != unique_indicators[i]]

final_df = final_df.drop(['Indicator'], axis=1)
for indicator in indicators:
  final_df[indicator] = 0
  final_df[indicator] = final_df[indicator].astype(float)

for country in df_unesco['Country'].unique():
  for year in df_unesco['Time'].unique():
    date_country_matches = df_unesco[(df_unesco.Country == country) & (df_unesco.Time == year)]
    # we need to use each indicator value here to construct the new single row
    for i, row in date_country_matches.iterrows():
      # matching on country, year set final_df indicator value for this row
      # check if row[indicator] is one that we want
      current_indicator = row.Indicator
      if current_indicator in indicators:
        #set final df at same year and country at row_indicator to value
        final_df.loc[(final_df.Time == row.Time) & (final_df.Country == row.Country), current_indicator] = row.Value
final_df.head()
#then we can merge with df_hap

DEMO_IND       object
Indicator      object
LOCATION       object
Country        object
TIME            int64
Time            int64
Value         float64
Flag Codes     object
Flags          object
dtype: object


DEMO_IND LOCATION Country  TIME  Time       Value Flag Codes  \
186335  NY_GDP_PCAP_CN      CAN  Canada  1970  1970  4307.00759        NaN   
186336  NY_GDP_PCAP_CN      CAN  Canada  1971  1971  4631.20116        NaN   
186337  NY_GDP_PCAP_CN      CAN  Canada  1972  1972  5089.68640        NaN   
186338  NY_GDP_PCAP_CN      CAN  Canada  1973  1973  5871.18762        NaN   
186339  NY_GDP_PCAP_CN      CAN  Canada  1974  1974  6888.51792        NaN   

       Flags  Fertility rate, total (births per woman)  \
186335   NaN                                     2.258   
186336   NaN                                     2.141   
186337   NaN                                     1.980   
186338   NaN                                     1.890   
186339   NaN                                     1.837   

        Life expectancy at birth, total (years)  ...  GNI (current LCU)  \
186335                                 72.70049  ...       9.049216e+10   
186336                                 73.02927  ...       9.877660e+10   
186337                                 72.93390  ...       1.104571e+11   
186338                                 73.16268  ...       1.295094e+11   
186339                                 73.23756  ...       1.545532e+11   

        GNI per capita, Atlas method (current US$)  \
186335                                      3970.0   
186336                                      4390.0   
186337                                      5050.0   
186338                                      6060.0   
186339                                      7190.0   

        GNI per capita, PPP (current international $)  \
186335                                            0.0   
186336                                            0.0   
186337                                            0.0   
186338                                            0.0   
186339                                            0.0   

        Population aged 14 years or younger   Population aged 15-24 years   \
186335                              6440.147                      3925.468   
186336                              6358.735                      4079.901   
186337                              6290.521                      4213.894   
186338                              6223.796                      4328.185   
186339                              6140.715                      4428.317   

        Population aged 25-64 years   Population aged 65 years or older   \
186335                      9293.993                            1714.718   
186336                      9521.813                            1763.011   
186337                      9758.405                            1809.374   
186338                     10007.184                            1856.157   
186339                     10270.428                            1906.443   

        Prevalence of HIV, total (% of population ages 15-49)  \
186335                                                0.0       
186336                                                0.0       
186337                                                0.0       
186338                                                0.0       
186339                                                0.0       

        Poverty headcount ratio at $1.90 a day (PPP) (% of population)  \
186335                                                0.0                
186336                                                0.0                
186337                                                0.0                
186338                                                0.0                
186339                                                0.0                

        Total population   
186335          21374.326  
186336          21723.460  
186337          22072.194  
186338          22415.322  
186339          22745.903  

[5 rows x 28 columns]

In [19]:
df_final = final_df.copy()

We elected to rename the features just to make the dataframe look simpler and cleaner. 

In [20]:
df_final = (df_final[["Country", "Time"] + indicators]).copy()
df_final.rename(columns={"Time": "Year",
                         "Fertility rate, total (births per woman)": "fertilityRate",
                         "Life expectancy at birth, total (years)": "lifeExpectancy",
                         "Mortality rate, infant (per 1,000 live births)": "mortalityRate",
                         "Population growth (annual %)": "popGrowth",
                         "Rural population (% of total population)": "ruralPopPct",
                         "GDP growth (annual %)": "gdpGrowthPct",
                         "GDP (current US$)": "gdpUS",
                         "GDP per capita (current US$)": "gdpPerCapitaUS",
                         "GDP per capita, PPP (current international $)": "gdpPerCapita ppp",
                         "GDP, PPP (current international $)": "gdp pppInternational",
                         "GNI (current LCU)": "gni",
                         "GNI per capita, Atlas method (current US$)": "gniPerCapita",
                         "GNI per capita, PPP (current international $)": "gniPerCapita ppp",
                         "Population aged 14 years or younger ": "pop14under",
                         "Population aged 15-24 years ": "pop15to24",
                         "Population aged 25-64 years ": "pop25to64",
                         "Population aged 65 years or older ": "pop65over",
                         "Prevalence of HIV, total (% of population ages 15-49)": "hivPct",
                         "Poverty headcount ratio at $1.90 a day (PPP) (% of population)": "povertyRatio",
                         "Total population ": "totalPop"},
                inplace=True
                )

In [21]:
df = df_final.copy()

In [27]:
len(df["Country"].unique())

212

We noticed that a lot of countries had a lot of slighly different naming conventions.  This cell properly realligns all country namings so that the two dataframes, the unesco dataframe and the happiness dataframe, can be merged effectively.

In [23]:
for i, row in df_hap.iterrows():
  if df_hap.at[i, "Country"] == "United States":
    df_hap.at[i, "Country"] = "United States of America"
  # need to do for all countries that don't allign

df_combined = df_final.merge(df_hap, on=["Country", "Year"])

In [28]:
len(df_final["Country"].unique())

212

Finally, we saved and downlaoded the two csv's below to avoid having to re-run somewhat expensive data cleaning steps.  
- "unesco.csv" is a csv where each observation is a country in a given year, for years 1970-2019.
- "unesco_train.csv" is a csv where each observation is a country in a given year, for years 2015-2019.  Additionally, a happiness score is associated with each observation.

In [ ]:
df_hap.to_csv("happiness.csv", index=False)
df_final.to_csv("unesco.csv", index=False)
df_combined.to_csv("unesco_train.csv", index=False)
#upload files to .csv

Changing Happiness Score to whole number

In [ ]:
import numpy as np
df_countries.loc[:,'Happiness Score']=df_countries['Happiness Score'].apply(np.round)


Using Naive Bayes Algorithm


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

learning_df = df_countries[df_countries['Time'].isin([2015, 2016, 2017])]
# display(learning_df)
X = learning_df.drop(["Happiness Score", "Country", "Time"], axis=1)
y = learning_df["Happiness Score"]
# display(X)
# display(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)
gnb = GaussianNB()

# display(X_train)
# display(y_train)
y_pred = gnb.fit(X_train, y_train).predict(X_test)
accuracy = (y_test == y_pred).sum() / len(y_test)
print("%d / %d correct Happiness Score predictions" % ((y_test == y_pred).sum(), X_test.shape[0]))
print("Bayesian Accuracy ", accuracy)

ValueError: ignored

Using Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = (y_test == y_pred).sum() / len(y_test)
print("%d / %d correct Happiness Score predictions" % ((y_test == y_pred).sum(), X_test.shape[0]))
print("Random Forest Accuracy ", accuracy)
importances = clf.feature_importances_
importances = pd.DataFrame({'Feature':X_train.columns,'Importance':importances})
graph = importances.sort_values(by='Importance').set_index('Feature')
graph.plot.bar(figsize=(30,10), rot=45)

In [ ]:
final_df